In [8]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

project_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/"
data_path = project_path + "data/"

# Load cleaned parquet
df = pd.read_parquet(data_path + "cleaned_data.parquet")

state_map = {
    "AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California",
    "CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia",
    "HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa",
    "KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland",
    "MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi",
    "MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire",
    "NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina",
    "ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania",
    "RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee",
    "TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington",
    "WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming","DC":"District Of Columbia"
}

df['State'] = df['State'].apply(lambda x: state_map.get(x, x))


df.shape


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(554719, 22)

In [9]:
df[['clean_text', 'Product', 'Issue', 'Sub-issue', 'State', 'date_received']].head(10)


,clean_text,Product,Issue,Sub-issue,State,date_received
0,military star card has violated my consumer ri...,DEBT COLLECTION,TOOK OR THREATENED TO TAKE NEGATIVE OR LEGAL A...,SEIZED OR ATTEMPTED TO SEIZE YOUR PROPERTY,Texas,2021-06-27
1,on xx xx year charged for a order using a clos...,"MONEY TRANSFER, VIRTUAL CURRENCY, OR MONEY SER...",FRAUD OR SCAM,NO_SUB_ISSUE,Connecticut,2025-09-09
2,dear cfpb i am sending to dispute fraudulent i...,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,INCORRECT INFORMATION ON YOUR REPORT,INFORMATION BELONGS TO SOMEONE ELSE,Florida,2025-08-04
3,i am writing to file a complaint against vy fe...,CHECKING OR SAVINGS ACCOUNT,PROBLEM CAUSED BY YOUR FUNDS BEING LOW,OVERDRAFTS AND OVERDRAFT FEES,New York,2025-01-19
4,these companies are in clear violation of the ...,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,IMPROPER USE OF YOUR REPORT,REPORTING COMPANY USED YOUR REPORT IMPROPERLY,North Carolina,2024-04-10
5,the list of inquries listed below are a result...,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",IMPROPER USE OF YOUR REPORT,REPORTING COMPANY USED YOUR REPORT IMPROPERLY,South Carolina,2023-07-18
6,this is my numerous request that i have been a...,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",INCORRECT INFORMATION ON YOUR REPORT,INFORMATION BELONGS TO SOMEONE ELSE,California,2023-01-05
7,upon reviewing my credit report i have identif...,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,INCORRECT INFORMATION ON YOUR REPORT,INFORMATION BELONGS TO SOMEONE ELSE,Texas,2025-04-22
8,this is not my account i have no idea who this...,DEBT COLLECTION,ATTEMPTS TO COLLECT DEBT NOT OWED,DEBT WAS PAID,New York,2022-02-06
9,my credit report should only reflect accurate ...,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,INCORRECT INFORMATION ON YOUR REPORT,ACCOUNT INFORMATION INCORRECT,New York,2025-06-06


In [10]:
df = df[df['clean_text'].str.len() > 20]
df = df[df['clean_text'].notna()].reset_index(drop=True)

df.shape


(554719, 22)

In [11]:
df['Product'].value_counts().head(20)


,count
Product,
CREDIT REPORTING OR OTHER PERSONAL CONSUMER REPORTS,285429
"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR OTHER PERSONAL CONSUMER REPORTS",116920
DEBT COLLECTION,51365
CHECKING OR SAVINGS ACCOUNT,27337
"MONEY TRANSFER, VIRTUAL CURRENCY, OR MONEY SERVICE",18370
CREDIT CARD,14904
CREDIT CARD OR PREPAID CARD,12129
MORTGAGE,10045
VEHICLE LOAN OR LEASE,6385


In [12]:
import pandas as pd

sampled_df = (
    df.groupby('Product', group_keys=False)
      .apply(lambda x: x.sample(frac=min(1, 40000/len(df)), random_state=42))
)

sampled_df = sampled_df.reset_index(drop=True)

sampled_df.shape


/tmp/ipython-input-2871666128.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=min(1, 40000/len(df)), random_state=42))


(40000, 22)

In [13]:
texts = sampled_df['clean_text'].tolist()
len(texts)


40000

In [14]:
!pip install sentence-transformers --quiet

from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

print("Embedding model loaded.")


Embedding model loaded.


In [15]:
import numpy as np

embeddings = embedder.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings.shape


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

(40000, 384)

In [16]:
!pip install bertopic[hdbscan] --quiet

from bertopic import BERTopic

topic_model = BERTopic(
    verbose=True,
    calculate_probabilities=False,
    low_memory=True,
    min_topic_size=100,  # prevents hundreds of tiny meaningless topics
)

topics, _ = topic_model.fit_transform(texts, embeddings)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 12.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.
2025-11-21 11:06:55,315 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-21 11:09:38,007 - BERTopic - Dimensionality - Completed ✓
2025-11-21 11:09:38,010 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-21 11:09:49,925 - BERTopic - Cluster - Completed ✓
2025-11-21 11:09:49,937 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-21 11:09:52,074 - BERTopic - Representation - Completed ✓


In [18]:
sampled_df['topic'] = topics
sampled_df.head()


,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,...,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,date_received,year,month_num,day,clean_text,topic
0,CHECKING OR SAVINGS ACCOUNT,Checking account,MANAGING AN ACCOUNT,DEPOSITS AND WITHDRAWALS,XX/XX/year> I got a text about my phone esim c...,None,"ACIMA CREDIT, LLC",New York,11236,None,...,Closed with explanation,Yes,N/A,11350330,2025-01-02,2025,1,2,xx xx year i got a text about my phone esim ch...,3
1,CHECKING OR SAVINGS ACCOUNT,Checking account,CLOSING AN ACCOUNT,COMPANY CLOSED YOUR ACCOUNT,On XX/XX/2022 I withdrew money from the bank A...,None,BANCO POPULAR DE PUERTO RICO,New York,11235,None,...,Closed with explanation,Yes,N/A,6473472,2023-01-24,2023,1,24,on xx xx i withdrew money from the bank atm an...,1
2,CHECKING OR SAVINGS ACCOUNT,Checking account,CLOSING AN ACCOUNT,COMPANY CLOSED YOUR ACCOUNT,On XX/XX/XXXX approximately {$9000.00}. was de...,None,"HUNTINGTON NATIONAL BANK, THE",Michigan,486XX,Older American,...,Closed with non-monetary relief,Yes,N/A,13134910,2025-04-23,2025,4,23,on xx xx approximately was deposited in a newl...,1
3,CHECKING OR SAVINGS ACCOUNT,Checking account,MANAGING AN ACCOUNT,PROBLEM USING A DEBIT OR ATM CARD,My Name is XXXX XXXX XXXX i am president of No...,None,JPMORGAN CHASE & CO.,California,95843,Older American,...,Closed with explanation,Yes,N/A,11475786,2025-01-11,2025,1,11,my me is i am president of nonprofit organizat...,-1
4,CHECKING OR SAVINGS ACCOUNT,CD (Certificate of Deposit),MANAGING AN ACCOUNT,FUNDS NOT HANDLED OR DISBURSED AS INSTRUCTED,Our CD interest check was mailed by XXXX in X...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,Minnesota,55311,Older American,...,Closed with explanation,Yes,N/A,7399767,2023-08-14,2023,8,14,our cd interest check was mailed by in of and ...,22


In [19]:
topic_counts_per_product = (
    sampled_df.groupby(['Product', 'topic'])
              .size()
              .reset_index(name='count')
              .sort_values(['Product', 'count'], ascending=[True, False])
)

topic_counts_per_product.head(20)


,Product,topic,count
2,CHECKING OR SAVINGS ACCOUNT,1,844
0,CHECKING OR SAVINGS ACCOUNT,-1,593
12,CHECKING OR SAVINGS ACCOUNT,22,175
18,CHECKING OR SAVINGS ACCOUNT,54,130
13,CHECKING OR SAVINGS ACCOUNT,24,123
15,CHECKING OR SAVINGS ACCOUNT,34,38
4,CHECKING OR SAVINGS ACCOUNT,3,23
17,CHECKING OR SAVINGS ACCOUNT,51,14
1,CHECKING OR SAVINGS ACCOUNT,0,5
7,CHECKING OR SAVINGS ACCOUNT,6,5


In [21]:
clean_root_causes = top_root_causes[top_root_causes['topic'] != -1]
clean_root_causes


,Product,topic,count
2,CHECKING OR SAVINGS ACCOUNT,1,844
12,CHECKING OR SAVINGS ACCOUNT,22,175
22,CREDIT CARD,1,300
25,CREDIT CARD,4,96
50,CREDIT CARD OR PREPAID CARD,1,348
53,CREDIT CARD OR PREPAID CARD,4,47
71,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,2,939
73,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,4,737
136,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",2,400
137,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",3,366


In [22]:
def get_keywords(topic_id):
    try:
        return ", ".join([word for word, _ in topic_model.get_topic(topic_id)[:5]])
    except:
        return ""

clean_root_causes['keywords'] = clean_root_causes['topic'].apply(get_keywords)
clean_root_causes


,Product,topic,count,keywords
2,CHECKING OR SAVINGS ACCOUNT,1,844,"bank, was, card, they, the"
12,CHECKING OR SAVINGS ACCOUNT,22,175,"wells, fargo, was, they, money"
22,CREDIT CARD,1,300,"bank, was, card, they, the"
25,CREDIT CARD,4,96,"late, payments, payment, on, account"
50,CREDIT CARD OR PREPAID CARD,1,348,"bank, was, card, they, the"
53,CREDIT CARD OR PREPAID CARD,4,47,"late, payments, payment, on, account"
71,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,2,939,"inquiries, inquiry, xx, hard, credit"
73,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,4,737,"late, payments, payment, on, account"
136,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",2,400,"inquiries, inquiry, xx, hard, credit"
137,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",3,366,"identity, accounts, theft, fraudulent, my"


In [24]:
def interpret_keywords_safe(kw: str):
    kw = kw.lower()
    # split into tokens so "car" is not found inside "card"
    tokens = [t.strip() for t in kw.replace(",", " ").split() if t.strip()]

    def has(*words):
        return any(w in tokens for w in words)

    # More precise rules
    if has("identity", "theft", "fraud", "fraudulent"):
        return "Identity theft and fraudulent accounts"

    if has("inquiries", "inquiry"):
        return "Hard inquiries / unauthorized inquiries"

    if has("late", "payments", "payment"):
        return "Late payments / payment reporting issues"

    if has("mortgage", "escrow"):
        return "Mortgage / escrow account issues"

    if has("zelle"):
        return "Zelle transaction disputes"

    if ("app" in tokens or "cashapp" in tokens or "cash" in tokens) and not has("zelle"):
        return "Cash App / mobile payment disputes"

    if has("car", "vehicle", "auto"):
        return "Vehicle loan / auto leasing issues"

    if (has("loans", "loan") and "student" in tokens) or ("student" in tokens and "loan" in tokens):
        return "Student loan servicing errors"

    if has("debt") and has("collection"):
        return "Debt collection practices"

    # fallback, but smarter: distinguish card vs bank vs generic
    if "card" in tokens:
        return "Card-related dispute or transaction issue"
    if "bank" in tokens:
        return "Bank account or transaction dispute"

    return "General account or transaction dispute"


# Recompute interpretations with the safe logic
clean_root_causes["interpretation"] = clean_root_causes["keywords"].apply(interpret_keywords_safe)

clean_root_causes[["Product", "topic", "keywords", "interpretation"]]


,Product,topic,keywords,interpretation
2,CHECKING OR SAVINGS ACCOUNT,1,"bank, was, card, they, the",Card-related dispute or transaction issue
12,CHECKING OR SAVINGS ACCOUNT,22,"wells, fargo, was, they, money",General account or transaction dispute
22,CREDIT CARD,1,"bank, was, card, they, the",Card-related dispute or transaction issue
25,CREDIT CARD,4,"late, payments, payment, on, account",Late payments / payment reporting issues
50,CREDIT CARD OR PREPAID CARD,1,"bank, was, card, they, the",Card-related dispute or transaction issue
53,CREDIT CARD OR PREPAID CARD,4,"late, payments, payment, on, account",Late payments / payment reporting issues
71,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,2,"inquiries, inquiry, xx, hard, credit",Hard inquiries / unauthorized inquiries
73,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,4,"late, payments, payment, on, account",Late payments / payment reporting issues
136,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",2,"inquiries, inquiry, xx, hard, credit",Hard inquiries / unauthorized inquiries
137,"CREDIT REPORTING, CREDIT REPAIR SERVICES, OR O...",3,"identity, accounts, theft, fraudulent, my",Identity theft and fraudulent accounts


In [25]:
%whos


Variable                   Type                   Data/Info
-----------------------------------------------------------
BERTopic                   type                   <class 'bertopic._bertopic.BERTopic'>
SentenceTransformer        type                   <class 'sentence_transfor<...>mer.SentenceTransformer'>
clean_root_causes          DataFrame                                       <...>ayment reporting issues  
data_path                  str                    /content/drive/MyDrive/Da<...>t_Trend_Forecasting/data/
df                         DataFrame                                       <...>554719 rows x 22 columns]
drive                      module                 <module 'google.colab.dri<...>s/google/colab/drive.py'>
embedder                   SentenceTransformer    SentenceTransformer(\n  (<...>})\n  (2): Normalize()\n)
embeddings                 ndarray                40000x384: 15360000 elems, type `float32`, 61440000 bytes (58.59375 Mb)
get_keywords               functio

In [26]:
root_cause_rows_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows.csv"

sampled_df.to_csv(root_cause_rows_path, index=False)
print("Saved:", root_cause_rows_path)


Saved: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows.csv


In [27]:
root_cause_summary_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv"

clean_root_causes.to_csv(root_cause_summary_path, index=False)
print("Saved:", root_cause_summary_path)


Saved: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv
